In [1]:
from dask.distributed import LocalCluster, Client
cluster = LocalCluster(n_workers=4,
                       threads_per_worker=4,
                       memory_target_fraction=0.95,
                       memory_limit='8GB')
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 29.80 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40365,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 29.80 GiB
Comm: tcp://127.0.0.1:44349,Total threads: 4
Dashboard: http://127.0.0.1:43033/status,Memory: 7.45 GiB
Nanny: tcp://127.0.0.1:38725,


In [2]:
import dask.dataframe as dd
import pandas as pd
import re

In [3]:
df = dd.read_csv("archive-cc-2014-nc.csv")
df

,archive_id,text_nc
npartitions=42,,
,object,object
,...,...
...,...,...
,...,...
,...,...


In [4]:
%%time
df.head()

CPU times: user 136 ms, sys: 61.4 ms, total: 198 ms
Wall time: 1.59 s


,archive_id,text_nc
0,CNNW_20110513_133540_CNN_Newsroom,been fascinate for me is how much he has a loy...
1,CNNW_20110518_223642_The_Situation_Room_With_W...,a deal to secure the release of dominique stra...
2,CNNW_20110525_230000_John_King_USA,"england, to the queen. >> reporter: one critic..."
3,CNNW_20110526_000000_In_the_Arena,tonight we have some amazing pictures to show ...
4,CNNW_20110526_010000_Piers_Morgan_Tonight,the verge of being indicted? i'll ask legal ea...


In [5]:
%%time
df[df.archive_id.isin(['CNNW_20110525_230000_John_King_USA', 'CNNW_20110526_010000_Piers_Morgan_Tonight'])].compute()

CPU times: user 778 ms, sys: 253 ms, total: 1.03 s
Wall time: 8.51 s


,archive_id,text_nc
2,CNNW_20110525_230000_John_King_USA,"england, to the queen. >> reporter: one critic..."
4,CNNW_20110526_010000_Piers_Morgan_Tonight,the verge of being indicted? i'll ask legal ea...


In [6]:
# Define a function for string replacement
def replace_strings(column):
    try:
        return column.apply(lambda x: x if pd.isna(x) else re.sub("b'(.*)'", '\\1', x))
    except Exception as e:
        return column

In [7]:
%%time
first = True
OUTPUT_FILE = '../shared/archive-cc-2014-all-nc.csv.gz'
for i, adf in enumerate(pd.read_csv('../shared/archive-cc-2014.csv.xz', chunksize=1000)):
    print(i)
    # FIXME: don't need for 2017
    #adf = adf.apply(lambda col: replace_strings(col), axis=0)
    idents = adf.identifier.tolist()
    cdf = df[df.archive_id.isin(idents)].compute()
    #print(len(cdf))
    adf = pd.merge(adf, cdf, how='left', left_on='identifier', right_on='archive_id')
    del adf['archive_id']
    if first:
        adf.to_csv(OUTPUT_FILE, index=False, header=first, compression='gzip')
        first = False
    else:
        adf.to_csv(OUTPUT_FILE, mode='a', index=False, header=first, compression='gzip')
    #if i >= 9:
    #    break

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27